In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 256
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.182477951049805 | KNN Loss: 6.232606410980225 | BCE Loss: 1.94987154006958
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.211150169372559 | KNN Loss: 6.232600688934326 | BCE Loss: 1.9785493612289429
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.215576171875 | KNN Loss: 6.232440948486328 | BCE Loss: 1.98313570022583
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.188455581665039 | KNN Loss: 6.232435703277588 | BCE Loss: 1.9560197591781616
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.175670623779297 | KNN Loss: 6.232313632965088 | BCE Loss: 1.9433574676513672
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.12234878540039 | KNN Loss: 6.232375621795654 | BCE Loss: 1.8899729251861572
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.198297500610352 | KNN Loss: 6.232083797454834 | BCE Loss: 1.9662132263183594
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.153447151184082 | KNN Loss: 6.232396602630615 | BCE Loss: 1.9210506677627

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.318770408630371 | KNN Loss: 6.189958572387695 | BCE Loss: 1.1288115978240967
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.308002471923828 | KNN Loss: 6.188998222351074 | BCE Loss: 1.119004487991333
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.333943843841553 | KNN Loss: 6.183051586151123 | BCE Loss: 1.1508922576904297
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.269381523132324 | KNN Loss: 6.177389144897461 | BCE Loss: 1.0919926166534424
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.314318656921387 | KNN Loss: 6.177396774291992 | BCE Loss: 1.136922001838684
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.283210754394531 | KNN Loss: 6.16886043548584 | BCE Loss: 1.1143503189086914
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.2974090576171875 | KNN Loss: 6.164889812469482 | BCE Loss: 1.1325193643569946
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.250821590423584 | KNN Loss: 6.15731143951416 | BCE Loss: 1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.800716876983643 | KNN Loss: 5.751204490661621 | BCE Loss: 1.0495123863220215
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.809625625610352 | KNN Loss: 5.739058017730713 | BCE Loss: 1.0705674886703491
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.784581184387207 | KNN Loss: 5.72567081451416 | BCE Loss: 1.0589104890823364
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.7950334548950195 | KNN Loss: 5.731474876403809 | BCE Loss: 1.0635583400726318
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.8036208152771 | KNN Loss: 5.738285064697266 | BCE Loss: 1.0653358697891235
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.782561779022217 | KNN Loss: 5.733485221862793 | BCE Loss: 1.0490766763687134
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.798057556152344 | KNN Loss: 5.735341548919678 | BCE Loss: 1.062716007232666
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.756974697113037 | KNN Loss: 5.7236127853393555 | BCE Loss:

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.715782642364502 | KNN Loss: 5.660604476928711 | BCE Loss: 1.0551780462265015
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.747676849365234 | KNN Loss: 5.711723327636719 | BCE Loss: 1.0359532833099365
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.675665378570557 | KNN Loss: 5.6262688636779785 | BCE Loss: 1.0493966341018677
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.724010467529297 | KNN Loss: 5.652440547943115 | BCE Loss: 1.0715696811676025
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.698491096496582 | KNN Loss: 5.65304708480835 | BCE Loss: 1.0454440116882324
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.679124355316162 | KNN Loss: 5.627102851867676 | BCE Loss: 1.0520216226577759
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.677981853485107 | KNN Loss: 5.628269672393799 | BCE Loss: 1.0497121810913086
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.7041120529174805 | KNN Loss: 5.648830890655518 | BCE Lo

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.689226150512695 | KNN Loss: 5.645828723907471 | BCE Loss: 1.0433974266052246
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.715989589691162 | KNN Loss: 5.688199520111084 | BCE Loss: 1.0277901887893677
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.751047134399414 | KNN Loss: 5.682516098022461 | BCE Loss: 1.0685312747955322
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.694040775299072 | KNN Loss: 5.638171195983887 | BCE Loss: 1.0558695793151855
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.681375503540039 | KNN Loss: 5.631553649902344 | BCE Loss: 1.0498217344284058
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.676664352416992 | KNN Loss: 5.613317966461182 | BCE Loss: 1.0633466243743896
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.730112075805664 | KNN Loss: 5.698916435241699 | BCE Loss: 1.0311956405639648
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.651851654052734 | KNN Loss: 5.611672878265381 | BCE Los

...

Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 6.671750068664551 | KNN Loss: 5.6568074226379395 | BCE Loss: 1.0149428844451904
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.816885471343994 | KNN Loss: 5.7464599609375 | BCE Loss: 1.0704253911972046
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.6697187423706055 | KNN Loss: 5.591679573059082 | BCE Loss: 1.0780394077301025
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.688450813293457 | KNN Loss: 5.608777046203613 | BCE Loss: 1.0796736478805542
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.66083288192749 | KNN Loss: 5.606150150299072 | BCE Loss: 1.054682731628418
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.629759311676025 | KNN Loss: 5.604778289794922 | BCE Loss: 1.0249810218811035
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.653360366821289 | KNN Loss: 5.59627628326416 | BCE Loss: 1.057084083557129
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 6.643348693847656 | KNN Loss: 5.606680870056152 | BCE L

Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 6.697253227233887 | KNN Loss: 5.628466606140137 | BCE Loss: 1.068786859512329
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.63754940032959 | KNN Loss: 5.617853164672852 | BCE Loss: 1.0196962356567383
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.67661190032959 | KNN Loss: 5.646628379821777 | BCE Loss: 1.0299835205078125
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.713959693908691 | KNN Loss: 5.652726650238037 | BCE Loss: 1.0612328052520752
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.686912536621094 | KNN Loss: 5.632821559906006 | BCE Loss: 1.054091215133667
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.7007598876953125 | KNN Loss: 5.641837120056152 | BCE Loss: 1.0589227676391602
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.736947059631348 | KNN Loss: 5.665217399597168 | BCE Loss: 1.0717295408248901
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 6.636678695678711 | KNN Loss: 5.590761661529541 | BC

Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 6.671650409698486 | KNN Loss: 5.602357387542725 | BCE Loss: 1.0692931413650513
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.641423225402832 | KNN Loss: 5.590216159820557 | BCE Loss: 1.0512068271636963
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.626206874847412 | KNN Loss: 5.591987609863281 | BCE Loss: 1.0342191457748413
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.725613594055176 | KNN Loss: 5.67105770111084 | BCE Loss: 1.0545557737350464
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.614880561828613 | KNN Loss: 5.602985858917236 | BCE Loss: 1.0118944644927979
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.7131028175354 | KNN Loss: 5.665530681610107 | BCE Loss: 1.0475720167160034
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.661980628967285 | KNN Loss: 5.599809646606445 | BCE Loss: 1.0621707439422607
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 6.632842063903809 | KNN Loss: 5.598044395446777 | BC

Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 6.684351921081543 | KNN Loss: 5.649083137512207 | BCE Loss: 1.0352685451507568
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.659478664398193 | KNN Loss: 5.603353977203369 | BCE Loss: 1.0561246871948242
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.66899299621582 | KNN Loss: 5.6213250160217285 | BCE Loss: 1.0476679801940918
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.660287857055664 | KNN Loss: 5.603072643280029 | BCE Loss: 1.0572152137756348
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.61958646774292 | KNN Loss: 5.595616340637207 | BCE Loss: 1.0239700078964233
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.686114311218262 | KNN Loss: 5.6205549240112305 | BCE Loss: 1.0655591487884521
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.647724628448486 | KNN Loss: 5.606989860534668 | BCE Loss: 1.0407347679138184
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 6.663987159729004 | KNN Loss: 5.633882522583008 | B

Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 6.70100212097168 | KNN Loss: 5.629997253417969 | BCE Loss: 1.0710049867630005
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.667936325073242 | KNN Loss: 5.608964920043945 | BCE Loss: 1.0589715242385864
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.705996513366699 | KNN Loss: 5.622093200683594 | BCE Loss: 1.0839033126831055
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.620012283325195 | KNN Loss: 5.596725940704346 | BCE Loss: 1.0232864618301392
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.694492816925049 | KNN Loss: 5.644543170928955 | BCE Loss: 1.0499496459960938
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.641535758972168 | KNN Loss: 5.601391792297363 | BCE Loss: 1.0401442050933838
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.67543363571167 | KNN Loss: 5.627509117126465 | BCE Loss: 1.0479243993759155
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 6.679417133331299 | KNN Loss: 5.596860408782959 | B

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.8217,  3.6125,  2.5152,  3.3619,  3.3702,  0.6665,  2.4555,  1.9713,
          2.2067,  1.9203,  2.0551,  2.0604,  0.7355,  1.7611,  1.2391,  1.4528,
          2.7113,  3.0813,  2.5865,  2.2356,  1.6618,  2.8783,  2.2024,  2.5269,
          2.3406,  1.6601,  2.0477,  1.3680,  1.4390,  0.3112, -0.2259,  0.9774,
          0.2020,  0.8970,  1.4721,  1.3503,  0.9529,  3.0087,  0.7396,  1.2688,
          0.9522, -0.7081, -0.2568,  2.2572,  2.0168,  0.6907, -0.2034,  0.0830,
          1.2786,  2.3213,  1.7738,  0.1577,  1.3278,  0.5104, -0.6213,  1.0817,
          1.4182,  1.2694,  1.2766,  1.7575,  0.5297,  0.8090,  0.1186,  1.6593,
          1.2432,  1.6040, -1.7948,  0.2945,  2.2321,  2.0777,  2.4627,  0.3653,
          1.2580,  2.2580,  1.9304,  1.2560,  0.2079,  0.7257,  0.2168,  1.5396,
          0.0038,  0.3593,  1.7438, -0.3736,  0.2129, -1.0330, -2.3198, -0.2471,
          0.5458, -1.7872,  0.4219, -0.1311, -0.5523, -0.8504,  0.5359,  1.2067,
         -0.6807, -0.6634,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 79.20it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 030 | Total loss: 9.630 | Reg loss: 0.014 | Tree loss: 9.630 | Accuracy: 0.000000 | 12.32 sec/iter
Epoch: 00 | Batch: 001 / 030 | Total loss: 9.622 | Reg loss: 0.013 | Tree loss: 9.622 | Accuracy: 0.000000 | 9.791 sec/iter
Epoch: 00 | Batch: 002 / 030 | Total loss: 9.615 | Reg loss: 0.012 | Tree loss: 9.615 | Accuracy: 0.000000 | 8.947 sec/iter
Epoch: 00 | Batch: 003 / 030 | Total loss: 9.607 | Reg loss: 0.011 | Tree loss: 9.607 | Accuracy: 0.000000 | 8.54 sec/iter
Epoch: 00 | Batch: 004 / 030 | Total loss: 9.600 | Reg loss: 0.010 | Tree loss: 9.600 | Accuracy: 0.005859 | 8.291 sec/iter
Epoch: 00 | Batch: 005 / 030 | Total loss: 9.592 | Reg loss: 0.010 | Tree loss: 9.592 | Accuracy: 0.019531 | 8.133 sec/iter
Epoch: 00 | Batch: 006 / 030 | Total loss: 9.584 | Reg loss: 0.009 | Tree loss: 9.584 | A

layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 02 | Batch: 000 / 030 | Total loss: 9.430 | Reg loss: 0.009 | Tree loss: 9.430 | Accuracy: 0.578125 | 8.098 sec/iter
Epoch: 02 | Batch: 001 / 030 | Total loss: 9.427 | Reg loss: 0.009 | Tree loss: 9.427 | Accuracy: 0.576172 | 8.081 sec/iter
Epoch: 02 | Batch: 002 / 030 | Total loss: 9.422 | Reg loss: 0.009 | Tree loss: 9.422 | Accuracy: 0.552734 | 8.064 sec/iter
Epoch: 02 | Batch: 003 / 030 | Total loss: 9.416 | Reg loss: 0.010 | Tree loss: 9.416 | Accuracy: 0.593750 | 8.047 sec/iter
Epoch: 02 | Batch: 004 / 030 | Total loss: 9.408 | Reg loss: 0.010 | Tree loss: 9.408 | Accuracy: 0.601562 | 8.031 sec/iter
Epoch: 02 | Batch: 005 / 030 | Total loss: 9.401 | Reg loss: 0.010 | Tree loss: 9.401 | Accuracy: 0.576172 | 7.986 sec/iter
Epoch: 02 | Batch: 006 / 030 | Total loss: 9.396 | Reg loss: 0.010 | Tree loss: 9.396 | Accuracy: 0.585938 | 7.94 sec/iter
Epoch: 02 | Batch: 007 / 030 | Total loss: 9.392

Epoch: 04 | Batch: 000 / 030 | Total loss: 9.251 | Reg loss: 0.015 | Tree loss: 9.251 | Accuracy: 0.560547 | 7.648 sec/iter
Epoch: 04 | Batch: 001 / 030 | Total loss: 9.238 | Reg loss: 0.015 | Tree loss: 9.238 | Accuracy: 0.585938 | 7.626 sec/iter
Epoch: 04 | Batch: 002 / 030 | Total loss: 9.225 | Reg loss: 0.015 | Tree loss: 9.225 | Accuracy: 0.605469 | 7.621 sec/iter
Epoch: 04 | Batch: 003 / 030 | Total loss: 9.219 | Reg loss: 0.015 | Tree loss: 9.219 | Accuracy: 0.570312 | 7.614 sec/iter
Epoch: 04 | Batch: 004 / 030 | Total loss: 9.201 | Reg loss: 0.015 | Tree loss: 9.201 | Accuracy: 0.589844 | 7.595 sec/iter
Epoch: 04 | Batch: 005 / 030 | Total loss: 9.186 | Reg loss: 0.016 | Tree loss: 9.186 | Accuracy: 0.595703 | 7.574 sec/iter
Epoch: 04 | Batch: 006 / 030 | Total loss: 9.179 | Reg loss: 0.016 | Tree loss: 9.179 | Accuracy: 0.564453 | 7.552 sec/iter
Epoch: 04 | Batch: 007 / 030 | Total loss: 9.150 | Reg loss: 0.016 | Tree loss: 9.150 | Accuracy: 0.603516 | 7.53 sec/iter
Epoch: 04

Epoch: 06 | Batch: 001 / 030 | Total loss: 8.816 | Reg loss: 0.021 | Tree loss: 8.816 | Accuracy: 0.578125 | 7.315 sec/iter
Epoch: 06 | Batch: 002 / 030 | Total loss: 8.819 | Reg loss: 0.021 | Tree loss: 8.819 | Accuracy: 0.568359 | 7.314 sec/iter
Epoch: 06 | Batch: 003 / 030 | Total loss: 8.783 | Reg loss: 0.021 | Tree loss: 8.783 | Accuracy: 0.574219 | 7.313 sec/iter
Epoch: 06 | Batch: 004 / 030 | Total loss: 8.783 | Reg loss: 0.021 | Tree loss: 8.783 | Accuracy: 0.572266 | 7.312 sec/iter
Epoch: 06 | Batch: 005 / 030 | Total loss: 8.733 | Reg loss: 0.021 | Tree loss: 8.733 | Accuracy: 0.593750 | 7.312 sec/iter
Epoch: 06 | Batch: 006 / 030 | Total loss: 8.719 | Reg loss: 0.022 | Tree loss: 8.719 | Accuracy: 0.583984 | 7.311 sec/iter
Epoch: 06 | Batch: 007 / 030 | Total loss: 8.683 | Reg loss: 0.022 | Tree loss: 8.683 | Accuracy: 0.580078 | 7.31 sec/iter
Epoch: 06 | Batch: 008 / 030 | Total loss: 8.636 | Reg loss: 0.023 | Tree loss: 8.636 | Accuracy: 0.601562 | 7.308 sec/iter
Epoch: 06

Epoch: 08 | Batch: 002 / 030 | Total loss: 8.262 | Reg loss: 0.025 | Tree loss: 8.262 | Accuracy: 0.570312 | 7.184 sec/iter
Epoch: 08 | Batch: 003 / 030 | Total loss: 8.246 | Reg loss: 0.026 | Tree loss: 8.246 | Accuracy: 0.574219 | 7.183 sec/iter
Epoch: 08 | Batch: 004 / 030 | Total loss: 8.200 | Reg loss: 0.026 | Tree loss: 8.200 | Accuracy: 0.566406 | 7.183 sec/iter
Epoch: 08 | Batch: 005 / 030 | Total loss: 8.150 | Reg loss: 0.026 | Tree loss: 8.150 | Accuracy: 0.593750 | 7.182 sec/iter
Epoch: 08 | Batch: 006 / 030 | Total loss: 8.137 | Reg loss: 0.026 | Tree loss: 8.137 | Accuracy: 0.593750 | 7.182 sec/iter
Epoch: 08 | Batch: 007 / 030 | Total loss: 8.100 | Reg loss: 0.026 | Tree loss: 8.100 | Accuracy: 0.595703 | 7.181 sec/iter
Epoch: 08 | Batch: 008 / 030 | Total loss: 8.053 | Reg loss: 0.027 | Tree loss: 8.053 | Accuracy: 0.560547 | 7.181 sec/iter
Epoch: 08 | Batch: 009 / 030 | Total loss: 8.004 | Reg loss: 0.027 | Tree loss: 8.004 | Accuracy: 0.597656 | 7.18 sec/iter
Epoch: 08

...

Epoch: 91 | Batch: 012 / 030 | Total loss: 1.026 | Reg loss: 0.039 | Tree loss: 1.026 | Accuracy: 0.580078 | 7.08 sec/iter
Epoch: 91 | Batch: 013 / 030 | Total loss: 1.000 | Reg loss: 0.039 | Tree loss: 1.000 | Accuracy: 0.591797 | 7.08 sec/iter
Epoch: 91 | Batch: 014 / 030 | Total loss: 0.981 | Reg loss: 0.039 | Tree loss: 0.981 | Accuracy: 0.593750 | 7.08 sec/iter
Epoch: 91 | Batch: 015 / 030 | Total loss: 0.983 | Reg loss: 0.039 | Tree loss: 0.983 | Accuracy: 0.576172 | 7.08 sec/iter
Epoch: 91 | Batch: 016 / 030 | Total loss: 0.959 | Reg loss: 0.040 | Tree loss: 0.959 | Accuracy: 0.591797 | 7.08 sec/iter
Epoch: 91 | Batch: 017 / 030 | Total loss: 0.983 | Reg loss: 0.040 | Tree loss: 0.983 | Accuracy: 0.541016 | 7.08 sec/iter
Epoch: 91 | Batch: 018 / 030 | Total loss: 0.920 | Reg loss: 0.040 | Tree loss: 0.920 | Accuracy: 0.630859 | 7.08 sec/iter
Epoch: 91 | Batch: 019 / 030 | Total loss: 0.937 | Reg loss: 0.040 | Tree loss: 0.937 | Accuracy: 0.560547 | 7.08 sec/iter
Epoch: 91 | Batc

Epoch: 93 | Batch: 013 / 030 | Total loss: 0.997 | Reg loss: 0.039 | Tree loss: 0.997 | Accuracy: 0.583984 | 7.063 sec/iter
Epoch: 93 | Batch: 014 / 030 | Total loss: 0.971 | Reg loss: 0.039 | Tree loss: 0.971 | Accuracy: 0.580078 | 7.062 sec/iter
Epoch: 93 | Batch: 015 / 030 | Total loss: 0.986 | Reg loss: 0.039 | Tree loss: 0.986 | Accuracy: 0.546875 | 7.061 sec/iter
Epoch: 93 | Batch: 016 / 030 | Total loss: 0.964 | Reg loss: 0.039 | Tree loss: 0.964 | Accuracy: 0.580078 | 7.061 sec/iter
Epoch: 93 | Batch: 017 / 030 | Total loss: 0.928 | Reg loss: 0.039 | Tree loss: 0.928 | Accuracy: 0.591797 | 7.06 sec/iter
Epoch: 93 | Batch: 018 / 030 | Total loss: 0.927 | Reg loss: 0.040 | Tree loss: 0.927 | Accuracy: 0.593750 | 7.059 sec/iter
Epoch: 93 | Batch: 019 / 030 | Total loss: 0.925 | Reg loss: 0.040 | Tree loss: 0.925 | Accuracy: 0.582031 | 7.058 sec/iter
Epoch: 93 | Batch: 020 / 030 | Total loss: 0.911 | Reg loss: 0.040 | Tree loss: 0.911 | Accuracy: 0.589844 | 7.057 sec/iter
Epoch: 93

Epoch: 95 | Batch: 014 / 030 | Total loss: 0.965 | Reg loss: 0.039 | Tree loss: 0.965 | Accuracy: 0.630859 | 7.018 sec/iter
Epoch: 95 | Batch: 015 / 030 | Total loss: 0.962 | Reg loss: 0.039 | Tree loss: 0.962 | Accuracy: 0.574219 | 7.017 sec/iter
Epoch: 95 | Batch: 016 / 030 | Total loss: 0.932 | Reg loss: 0.039 | Tree loss: 0.932 | Accuracy: 0.601562 | 7.016 sec/iter
Epoch: 95 | Batch: 017 / 030 | Total loss: 0.933 | Reg loss: 0.039 | Tree loss: 0.933 | Accuracy: 0.583984 | 7.015 sec/iter
Epoch: 95 | Batch: 018 / 030 | Total loss: 0.935 | Reg loss: 0.039 | Tree loss: 0.935 | Accuracy: 0.599609 | 7.015 sec/iter
Epoch: 95 | Batch: 019 / 030 | Total loss: 0.950 | Reg loss: 0.039 | Tree loss: 0.950 | Accuracy: 0.509766 | 7.014 sec/iter
Epoch: 95 | Batch: 020 / 030 | Total loss: 0.908 | Reg loss: 0.040 | Tree loss: 0.908 | Accuracy: 0.585938 | 7.013 sec/iter
Epoch: 95 | Batch: 021 / 030 | Total loss: 0.885 | Reg loss: 0.040 | Tree loss: 0.885 | Accuracy: 0.607422 | 7.012 sec/iter
Epoch: 9

Epoch: 97 | Batch: 015 / 030 | Total loss: 0.979 | Reg loss: 0.039 | Tree loss: 0.979 | Accuracy: 0.582031 | 6.964 sec/iter
Epoch: 97 | Batch: 016 / 030 | Total loss: 0.941 | Reg loss: 0.039 | Tree loss: 0.941 | Accuracy: 0.587891 | 6.963 sec/iter
Epoch: 97 | Batch: 017 / 030 | Total loss: 0.961 | Reg loss: 0.039 | Tree loss: 0.961 | Accuracy: 0.560547 | 6.962 sec/iter
Epoch: 97 | Batch: 018 / 030 | Total loss: 0.907 | Reg loss: 0.039 | Tree loss: 0.907 | Accuracy: 0.628906 | 6.961 sec/iter
Epoch: 97 | Batch: 019 / 030 | Total loss: 0.923 | Reg loss: 0.039 | Tree loss: 0.923 | Accuracy: 0.580078 | 6.96 sec/iter
Epoch: 97 | Batch: 020 / 030 | Total loss: 0.918 | Reg loss: 0.039 | Tree loss: 0.918 | Accuracy: 0.546875 | 6.959 sec/iter
Epoch: 97 | Batch: 021 / 030 | Total loss: 0.893 | Reg loss: 0.040 | Tree loss: 0.893 | Accuracy: 0.615234 | 6.958 sec/iter
Epoch: 97 | Batch: 022 / 030 | Total loss: 0.882 | Reg loss: 0.040 | Tree loss: 0.882 | Accuracy: 0.591797 | 6.956 sec/iter
Epoch: 97

Epoch: 99 | Batch: 016 / 030 | Total loss: 0.961 | Reg loss: 0.039 | Tree loss: 0.961 | Accuracy: 0.548828 | 6.894 sec/iter
Epoch: 99 | Batch: 017 / 030 | Total loss: 0.945 | Reg loss: 0.039 | Tree loss: 0.945 | Accuracy: 0.578125 | 6.893 sec/iter
Epoch: 99 | Batch: 018 / 030 | Total loss: 0.923 | Reg loss: 0.039 | Tree loss: 0.923 | Accuracy: 0.601562 | 6.892 sec/iter
Epoch: 99 | Batch: 019 / 030 | Total loss: 0.914 | Reg loss: 0.039 | Tree loss: 0.914 | Accuracy: 0.580078 | 6.891 sec/iter
Epoch: 99 | Batch: 020 / 030 | Total loss: 0.918 | Reg loss: 0.039 | Tree loss: 0.918 | Accuracy: 0.570312 | 6.89 sec/iter
Epoch: 99 | Batch: 021 / 030 | Total loss: 0.878 | Reg loss: 0.039 | Tree loss: 0.878 | Accuracy: 0.597656 | 6.889 sec/iter
Epoch: 99 | Batch: 022 / 030 | Total loss: 0.889 | Reg loss: 0.039 | Tree loss: 0.889 | Accuracy: 0.585938 | 6.887 sec/iter
Epoch: 99 | Batch: 023 / 030 | Total loss: 0.874 | Reg loss: 0.040 | Tree loss: 0.874 | Accuracy: 0.595703 | 6.886 sec/iter
Epoch: 99

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 11.736542962219993


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 2991


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

============== Pattern 665 ==============
============== Pattern 666 ==============
============== Pattern 667 ==============
============== Pattern 668 ==============
============== Pattern 669 ==============
============== Pattern 670 ==============
============== Pattern 671 ==============
============== Pattern 672 ==============
============== Pattern 673 ==============
============== Pattern 674 ==============
============== Pattern 675 ==============
============== Pattern 676 ==============
============== Pattern 677 ==============
============== Pattern 678 ==============
============== Pattern 679 ==============
============== Pattern 680 ==============
============== Pattern 681 ==============
============== Pattern 682 ==============
============== Pattern 683 ==============
============== Pattern 684 ==============
============== Pattern 685 ==============
============== Pattern 686 ==============
============== Pattern 687 ==============
============== Pattern 688 =======

============== Pattern 2165 ==============
============== Pattern 2166 ==============
============== Pattern 2167 ==============
============== Pattern 2168 ==============
============== Pattern 2169 ==============
============== Pattern 2170 ==============
============== Pattern 2171 ==============
============== Pattern 2172 ==============
============== Pattern 2173 ==============
============== Pattern 2174 ==============
============== Pattern 2175 ==============
============== Pattern 2176 ==============
============== Pattern 2177 ==============
============== Pattern 2178 ==============
============== Pattern 2179 ==============
============== Pattern 2180 ==============
============== Pattern 2181 ==============
============== Pattern 2182 ==============
============== Pattern 2183 ==============
============== Pattern 2184 ==============
============== Pattern 2185 ==============
============== Pattern 2186 ==============
============== Pattern 2187 ==============
===========

============== Pattern 2664 ==============
============== Pattern 2665 ==============
============== Pattern 2666 ==============
============== Pattern 2667 ==============
============== Pattern 2668 ==============
============== Pattern 2669 ==============
============== Pattern 2670 ==============
============== Pattern 2671 ==============
============== Pattern 2672 ==============
============== Pattern 2673 ==============
============== Pattern 2674 ==============
============== Pattern 2675 ==============
============== Pattern 2676 ==============
============== Pattern 2677 ==============
============== Pattern 2678 ==============
============== Pattern 2679 ==============
============== Pattern 2680 ==============
============== Pattern 2681 ==============
============== Pattern 2682 ==============
============== Pattern 2683 ==============
============== Pattern 2684 ==============
============== Pattern 2685 ==============
============== Pattern 2686 ==============
===========